# Introduction to Model Context Protocol

LLMs are limited by their training data and lack access to real-time information or specialized tools.

The **Model Context Protocol (MCP)** enables AI models to connect with external data sources, tools, and envrionments, allowing for the seamless transfer of information and capabilities between AI systems and the broader world.

## Terminology

Just as USB-C provides a standardized physical and logical interface for connecting various peripherals to computing devices, MCP offers a consistent protocol for linking AI models to external capabilities. This standardization beneftis the entire ecosystem:
- Users enjoy simpler and more consistent experiences across AI applications.
- AI application developers gain easy integration with a growing ecosystem of tools and data sources
- Tool and data providers need only create a single implementation that works with multiple AI applications
- The broader ecosystem benefits from increased interoperability, innovation, and reduced fragmentation

The **MxN integration problem** refers to the challenge of connecting M different AI applications to N different external tools or data sources without a standardized approach.

Without a protocol like MCP, developers would need to create MxN custom integrations - one for each possible pairing of an AI application with an external capability, which causes a lot of frictions and high mantainenance costs.

MCP transforms this into an **M+N** problem by providing a standard interface:
- *each AI application implements the client side of MCP once, and*
- *each tool/data source implements the server side one*.

Therefore, MCP is a standard like HTTP or USB-C, and is a protocol for connecting AI applications to external tools and data sources.

Just like client server relationships in HTTP, MCP has a client and a server.
- **Host** - the user-facing AI applications that end-users interact with directly. Examples include Anthropic's Claude Desktop, AI-enhanced IDEs like Cursor, inference libraries like HuggingFace Python SDK, or custom applications built in libraries LangChain or smolagents. Hosts initiate connections to MCP Servers and orchestrate the overall flow between user requests, LLM processing, and external tools.

- **Client** - a component within the host application that manages communication with a specific MCP Server. Each client maintains a 1-to-1 connection with a single Server, handling the protocol-level details of MCP communication and acting as an intermediary between the Host's logic and the external Server.

- **Server** - an external program or service that exposes capabilities (tools, resources, prompts) via the MCP protocol.

The value of AI applications relies on the capabilities the applications could offer:

| Capability | Description | Example |
| --- | --- | --- |
| Tools | Executable functions that the AI model can invoke to perform actions or retrieve computed data. Typically relating to the use case of the application. | A tool for a weather application might be a function that returns the weather in a specific location. |
| Resources | Read-only data sources that provide context without significant computation. | A researcher assistant might have a resource for scientific papers. |
| Prompts | Pre-defined templates or workflows that guide interactions between users, AI models, and the available capabilities. | A summarization prompt. |
| Sampling | Server-initiated requests for the Client/Host to perform LLM interactions, enabling recursive actions where the LLM can review generated content and make further decisions. | A writing application reviewing its own output and decide to refine it further. |

We may use their MCP entities in the following ways:

| Entity | Name | Description |
| --- | --- | --- |
| Tool | Code Interpreter | A tool that can execute code that the LLM writes. |
| Resource | Documentation | A resource that contains the documentation of the application. |
| Prompt | Code Style | A prompt that guides the LLM to generate code. |
| Sampling | Code Review | A sampling that allows the LLM to review the code and make further decisions. |

## Architectural Components of MCP

The MCP protocol is built on a client-server architecture that enables structured communication between AI models and external systems.

The MCP architecture consists of three primary components: Host, Client, and Server.

### Host

The **Host** is the user-facing AI application that end-users interact with directly.

For example
- AI Chat apps like OpenAI ChatGPT or Anthropic's Claude Desktop
- AI-enhanced IDEs like Cursor, or integraitons to tools like Continue.dev
- Custom AI agents and applications built in libraries like LangChain or smolagents

Responsibilities of the Host:
- managing user interactions and permissions
- initiating connections to MCP Servers via MCP Clients
- orchestrating the overall flow between user requests, LLM processing, and external tools
- rendering results back to users in a coherent format


In most cases, users will select their host application based on their needs and preferences.

### Client

The **Client** is a component within the Host application that manages communication with a specific MCP Server,
- Each Client maintains a 1-to-1 connection with a single Server
- Handles the protocol-level details of MCP communication
- Acts as the intermediary between the Host's logic and the external Server

### Server

The **Server** is an external program or service that exposes capabilities to AI models via the MCP protocol.

Servers
- provide access to specific external tools, data sources, or services
- Act as lightweight wrappers around existing functionality
- Cuan run locally or remotely
- Expose their cacpabilities in a standarddized format that Clients can discover and use

### Communication Flow

How the Host, Client, and Server interact in a MCP workflow:
- **User Interaction** - The user interacts with the **Host** application, expressing an intent or query.
- **Host Processing** - The **Host** processes the user's input, potentially using an LLM to understand the request and determine which external capabilities might be needed.
- **Client Connection** - The **Host** directs its **Client** component to connect to the appropriate **Server(s)**.
- **Capability Discovery** - The **Client** queries the **Server** to discover what capabilities (Tools, Resources, Prompts) it offers.
- **Capability Invocation** - Based on the user's needs or the LLM's determination, the **Host** instructs the **Client** to invoke specific capabilities from the **Server**.
- **Server Execution** - The **Server** executes the requested funcationality and returns results to the **Client**.
- **Result Integration** - The **Client** relays these results back to the **Host**, which incorporates them into the context for the LLM or presents them directly to the user.


A key advantage of this architecture is its modularity. A single **Host** can connect to multiple **Servers** simultaneously via different **Clients**. New **Servers** can be added to the ecosystem without requiring changes to existing **Hosts**. Capabilities can be easily composed across different **Servers**. This modularity transforms the traditional MxN integration problem into a more manageable M+N problem.

## The Communication Protocol

MCP defines a standardized communication protocol that enables **Clients** and **Servers** to exchange messages in a consistent, predictable way. This standardization is critical for interoperability across the community.

### JSON-RPC: The Foundation

At its core, MCP uses **JSON-RPC 2.0** as the message format for all communication between **Clients** and **Servers**.

JSON-RPC is a lightweight remote procedure call protocol encoded in JSON, which makes it
- Human-readable and easy to debug
- Language-agnostic, supporting implementation in any programming environment
- Well-established, with clear specifications and widespread adoption


```
Client ----[    Request     ]---> Server
       <---[    Response    ]----
       <---[  Notification  ]----
```

The protocol defines three types of meesages as shown in the diagram above.

**1. Requests**

-- Sent from **Client** to **Server** to initiate an operation. A **Request** message includes:
- A unique identifier (`id`)
- The method name to invoke (e.g., `tools/call`)
- Parameters for the method (if any)

Example Request:

```json
{
    "jsonrpc": "2.0",
    "id": 1,
    "method": "tools/call",
    "params": {
        "name": "weather",
        "arguments" {
            "location": "Houston"
        }
    }
}
```


**2. Responses**

-- Sent from **Server** to **Client** in reply to a Request. A **Response** message includes:
- The same `id` as the corresponding Request
- Either a `result` (for success) or an `error` (for failure)

Example Success Response:

```json
{
    "jsonrpc": "2.0",
    "id": 1,
    "result": {
        "temperature": 90,
        "conditions": "Sunny"
    }
}
```

Example Error Response:

```json
{
    "jsonrpc": "2.0",
    "id": 1,
    "error": {
        "code" -32602,
        "message": "Invalid location parameter"
    }
}
```



**3. Notifications**

-- One-way messages that don't require a response. Typically sent from **Server** to **Client** to provide updates or notifications about events.

Example Notification:
```json
{
    "jsonrpc": "2.0",
    "method": "progress",
    "params": {
        "message": "Processing data...",
        "percent": 50
    }
}
```

### Transport Mechanisms

The JSON-RPC defines the message format, but MCP also specifies how these messages are transported between **Clients** and **Servers**. There are two primary transport mechanisms:

**stdio (Standard Input/Output)**

The stdio transport is used for local communication, where the Client and Server run on the same machine:
- The **Host** application launches the **Server** as a subprocess and communicates with it by writing to its standard input (stdin) and reading from its standard output (stdout).
- Use cases: local tools like file system access or running local scripts
- Advantages: simple, not network configuration required, and securely sandboxed by the operating system.


**HTTP + SSE (Server-Sent Events) / Streamable HTTP**

The HTTP+SSE transport is used for remote communication, where the Client and Server might be on different machines:
- Communication happens over HTTP, with the **Server** using Server-Sent Events (SSE) to push updates to the **Client** over a persistent connection.
- Use cases: connecting to remote APIs, cloud services, or shared resources.
- Advantages: work across networks, enables integration with web services, and compatible with serverless environments.

**Streamable HTTP** offers more flexibility by allowing servers to dynamically upgrade to SSE for streaming when needed, while maintaining compatbility with serverless environments.

### The Interaction Lifecycle

The MCP protocol defines a structured interaction lifecycle between Clients and Servers:

**Initialization**

The Client connects to the Server and they exchange protocol versions and capabilities, and the Server responds with its supported protocol version and capabilities.

The Client confirms the initialization is complete via a notification message.

**Discovery**

The Client requests information about available capabilities and the Server responds with a list of available tools.

This process could be repeated for each tool, resource, or prompt type.

**Execution**

The Client invokes capabilities based on the Host's needs.

**Termination**

The connection is gracefully closed when no longer needed and the Server acknowledges the shutdown request.

The Client sends the final exit message to complete the termination.

## Understanding MCP Capabilities

MCP Servers expose a variety of capabilities to Clients through the communication protocol. These capabilities fall into four main categories, each with distinct characteristics and use cases.

### Tools

**Tools** are executable functions or actions that the AI models can invoke through the MCP protocol:
- **Control*** - Tools are typically **model-controlled**, meaning that the AI models decide when to call them based on the user's request and context.
- **Safety** - Due to their ability to perform actions with side effects, tool execution can be dangerous. Therefore, they typically require explicit user approval.
- **Use Cases** - Sending messages, creating tickets, querying APIs, performing calculations.


Example: A weather tool that fetches current weather data for a given location:

In Python
```python
def get_weather(location: str) -> dict:
    """Get the current weather for a specified location."""
    # Connect to weather API and fetch data
    return {
        "temperature": 90,
        "conditions": "Sunny",
        "humidity": 45
    }
```

In JavaScript
```javascript
function getWeather(location) {
    //Connect to weather API and fetch data
    return {
        temperature: 90,
        conditions: "Sunny",
        humidity: 45
    };
}
```

### Resources

**Resources** provide read-only access to data sources, allowing the AI models to retrieve context without executing complex logic:
- **Control** - Resources are **application-controlled**, meaning that the **Host** applications typically decide when to access them.
- **Nature** - They are designed for data retrieval with minimal computation, similar to GET endpoints in REST APIs.
- **Safety** - Since they are read-only, they typically present lower security risks than **Tools**.
- **Use Cases** - Accessing file contents, retrieving database records, reading configuration information.


Example: A resource that provides access to file content:

In Python
```python
def read_file(filepath: str) -> str:
    """Read the contents of a file at the specified path."""
    with open(filepath, 'r') as f:
        return f.read()
```


In JavaScript:
```javascript
function readFile(filepath) {
    // Using fs.readFile to read file contents
    const fs = require('fs');
    return new Promise((resolve, reject) => {
        fs.readFile(filepath, 'utf8', (err, data) => {
            if (err) {
                reject(err);
                return;
            }
            resolve(data);
        });
    });
}
```

### Prompts

**Prompts** are predefined templates or workflows that guide the interaction between the user, the AI model, and the Server's capabilities:
- **Control** - Prompts are **user-controlled**, often presented as options in the **Host** application's UI.
- **Purpose** - They structure interactions for optimal use of available **Tools** and **Resources**.
- **Selection** - Users typically select a prompt before the AI models begin processing, setting context for the interaction.
- **Use Cases** - Common workflows, specialized task templates, guided interactions.



Example: A prompt template for generating a code review:

In Python
```python
def code_review(code: str, language: str) -> list:
    """Generate a code review for the provided code snippet."""
    return [
        {
            "role": "system",
            "content": f"You are a code reviewer examining {language} code. Provide a detailed review highlighting best practices, potential issues, and suggestions for improvement."
        },
        {
            "role": "user",
            "content": f"Please review this {language} code:\n\n```{language}\n{code}\n```"
        {
    ]

```


In JavaScript
```javascript
function codeReview(code, language) {
    return [
        {
            role: 'system',
            content: `You are a code reviewer examining ${language} code. Provide a detailed review highlighting best practices, potential issues, and suggestions for improvement.`
        },
        {
            role: 'user',
            content: `Please review this ${language} code:\n\n\`\`\`${language}\n${code}\n\`\`\``
        }
    ];
}

```

### Sampling

**Sampling** allows **Servers** to request the **Client** (specifically, thoe **Host** application) to perform LLM interactions:
- **Control** - Sampling is **server-initiated** but requires Client/Host facilitation.
- **Purpose** - It enables server-driven agentic behaviors and potentially recursive or multi-step interactions.
- **Safety** - Like Tools, sampling operations typically require user approval.
- **Use cases** - Complex multi-step tasks, autonomous agent workflows, interactive processes.


Example: A Server might request the Client to analyze data it has processed:

In Python
```python
def request_sampling(messages, system_prompt=None, include_context="none"):
    """Request LLM sampling from the client."""
    # In real implementation, this would send a request to the client
    return {
        "role": "assistant",
        "content": "Analysis of the provided data..."
    }
```


In JavaScript
```javascript
function requestSampling(messages, systemPrompt = null, includeContext = "none") {
    // In real implementation, this would send a request to the client
    return {
        role: "assistant",
        content: "Analysis of the provided data..."
    };
}

function handleSamplingRequest(request) {
    const { messages, systemPrompt, includeContext } = request;

    return {
        role: 'assistant',
        content: 'Response to the sampling request...'
    };
}    
```


The sampling flow follows the following steps:
1. Server sends a `sampling/createMessaage` request to the Client
2. Client reviews the request and can modify it
3. Client samples from an LLM
4. Client reviews the completion
5. Client returns the result to the Server


NOTE: **This human-in-the-loop design ensures users maintain control over what the LLM sees and generates. When implementing sampling, it is important to provide clear, well-structured prompts and include relevant context.**

### How These Capabilities Work Together

| Capability | Controlled By | Direction | Side Effects | Approval Needed | Typical Use Cases |
| --- | --- | --- | --- | --- | --- |
| Tools | Model (LLM) | Client -> Server | Yes (potentially) | Yes | Actions, API calls, data manipulation |
| Resources | Application | Client -> Server | No (read-only) | Typically no | Data retrieval, context gathering |
| Prompts | User | Server -> Client | No | No (selected by user) | Guided workflows, specialized templates |
| Sampling | Server | Server -> Client -> Server | Indirectly | Yes | Multi-step tasks, agentic behaviors |


- A user might select a **Prompt** to start a specialized workflow
- The Prompt might include context from **Resources**
- During processing, the AI model might call **Tools** to perform specific actions
- For complex operations, the Server might use **Sampling** to request additional LLM processing

### Discoery Process

When a Client connects to a Server, it can query the available Tools, Resources, and Prompts through specific list methods:
- `tools/list` - Discover available Tools
- `resources/list` - Discover available Resources
- `prompts/list` - Discover available Prompts

This dynamic discovery mechanism allows Clients to adapt to the specific capabilities each Server offers without requiring hardcoded knowledge of the Server’s functionality.

## MCP SDK

The Model Context Protocol provides official SDKs for both JavaScript, Python, and other languages. which makes it easy to implement MCP clients and servers in our applications.


Both SDKs provides similar core functionality, following the MCP protocol specification we discussed in the previous section so that they can handle:
- Protocol-level communication
- Capability registration and discovery
- Message serialization/deserialziation
- Conection management
- Error handling

### Core Primitives Implementation

For [Python SDK](https://github.com/modelcontextprotocol/python-sdk), make sure to install the package first:

In [ ]:
!pip install "mcp[cli]"

In [ ]:
# Python Script for server.py

from mcp.server.fastmcp import FastMCP

# Create an MCP server
mcp = FastMCP("Weather Service")


@mcp.tools()
def get_weather(location: str) -> str:
    """Get the current weather for a specified location."""
    return f"Weather in {location}: Sunny, 90 degrees"


@mcp.resource("weather://{location}")
def weather_resource(location: str) -> str:
    """Provide weather data as a resource."""
    return f"Weather data for {location}: Sunny, 90 degrees."


@mcp.prompts()
def weather_report(location: str) -> str:
    """Create a weather report prompt."""
    return f"You are a weather reporter. Weather report for {location}?"


# Run the server
if __name__ == "__main__":
    mcp.run()

Once we saved this as a standalone `server.py` file, we can start it by running the server script:
```bash
mcp dev server.py
```

This will initialize a development server running the file `server.py` and log the following output:
```bash
Starting MCP inspector...
⚙️ Proxy server listening on port 6277
Spawned stdio transport
Connected MCP client to backing server transport
Created web app transport
Set up MCP proxy
🔍 MCP Inspector is up and running at http://127.0.0.1:6274 🚀
```

We can open the MCP inspector at http://127.0.0.1:6274 to see the server's capability and interact with them.

For JavaScript, we need to install
```bash
npm install @modelcontextprotocol/sdk
```

and then implement the following script
```javascript
import { McpServer, ResourceTemplate } from "@modelcontextprotocol/sdk/server/mcp.js";
import { StdioServerTransport } from "@modelcontextprotocol/sdk/server/stdio.js";
import { z } from "zod";

// Create an MCP server
const server = new McpServer({
    name: "Weather Service",
    version: "1.0.0"
});

// Tool implementation
server.tool("get_weather",
    { location: z.string() },
    async ({ location }) => ({
        content: [{
            type: "text",
            text: `Weather in ${location}: Sunny, 72°F`
        }]
    })
);

// Resource implementation
server.resource(
    "weather",
    new ResourceTemplate("weather://{location}", { list: undefined }),
    async (uri, { location }) => ({
        contents: [{
            uri: uri.href,
            text: `Weather data for ${location}: Sunny, 72°F`
        }]
    })
);

// Prompt implementation
server.prompt(
    "weather_report",
    { location: z.string() },
    async ({ location }) => ({
        messages: [
            {
                role: "assistant",
                content: {
                    type: "text",
                    text: "You are a weather reporter."
                }
            },
            {
                role: "user",
                content: {
                    type: "text",
                    text: `Weather report for ${location}?`
                }
            }
        ]
    })
);

// Run the server
const transport = new StdioServerTransport();
await server.connect(transport);
```

## MCP Clients

MCP Clients act as the bridge between AI applications (Hosts) and external capabilities provided by MCP Servers.

For example, Anthropic's Claude Desktop is an MCP Client, providing integration with various MCP Servers.

Cursor's MCP Client enables AI-powered coding assistance through direct integration with code editing capabilities. It supports multiple MCP Server connections and provides real-time tool invocation during coding.

### Configuring MCP Clients

MCP Hosts use configuration files to manage server connections. These files define which servers are available and how to connect to them.

The standard configuration file for MCP is named `mcp.json`. The basic structure of `mcp.json` looks like:
```json
{
    "servers": [
        {
            "name": "Server Name",
            "transport": {
                "type": "stdio|sse",
                // Transport-specific configuration
            }
        }
    ]
}

```
Here we have a single server with a name and a transport type. The transport type is either `stdio` or `sse`.

#### Configuration for stdio transfport

For local servers using stdio transport, the configuration includes the command and arguments to launch the server process:
```json
{
    "server": [
        {
            "name": "File Explorer",
            "transport": {
                "type": "stdio",
                "command": "python",
                "args": ["/path/to/file_explorer_server.py"]
            }
        }
    ]
}
```
Here we have a server called `"File Explorer"` that is a local script.

#### Configuration for HTTP+SSE transport

For remote servers using HTTP+SSE transport, the configuration includes the server URL:
```json
{
    "servers": [
        {
            "name": "Remote API Server",
            "transport": {
                "type": "sse",
                "url": "https://example.com/mcp-server"
            }
        }
    ]
}
```

#### Environment variables in configuration

Environment variables can be passed to server processes using the `env` field.

In Python, we can use the `os` module to access environment variables:
```python
import os

# Access environment variables
github_token = os.environ.get("GITHUB_TOKEN")
if not github_token:
    raise ValueError("GITHUB_TOKEN environment variable is required")

# Use the token in our server code
def make_github_request():
    headers = {"Authorization": f"Bearer {github_tokne}"}
    # ... rest of code
```

In JavaScript, we will use the `process.env` object to access environment variables:
```javascript
// Access environment variables
const githubToken = process.env.GITHUB_TOKEN;
if (!githubToken) {
    throw new Error("GITHUB_TOKEN environment variable is required");
}

// Use the token in your server code
function makeGithubRequest() {
    const headers = { "Authorization": `Bearer ${githubToken}` };
    // ... rest of your code
}
```

Next, we can implement the configuration `mcp.json` file:
```json
{
    "servers": [
        {
            "name": "GitHub API",
            "transport": {
                "type": "stdio",
                "command": "python",
                "args": ["/path/to/github.server.py"]
                "env": {
                    "GITHUB_TOKEN": "our_github_token"
                }
            }
        }
    ]
}
```

#### Configuration examples

In local server configuration, we have a local server that is a Python script which could be a file explorer or a code editor.
```json
{
  "servers": [
    {
      "name": "File Explorer",
      "transport": {
        "type": "stdio",
        "command": "python",
        "args": ["/path/to/file_explorer_server.py"]
      }
    }
  ]
}
```

In remote server configuration, we have a remote server that is a weather API.
```json
{
  "servers": [
    {
      "name": "Weather API",
      "transport": {
        "type": "sse",
        "url": "https://example.com/mcp-server"
      }
    }
  ]
}
```

### Code Clients

In `smolagents`, we can use the `ToolCollection` class to automatically discover and register tools from an MCP server. This is done by passing the `StdioServerParameter` or `SSEServerParameters` to the `ToolCollection.from_mcp` method.

In [ ]:
from smolagents import ToolCollection, CodeAgent
from mcp.client.stdio import StdioServerParameters

server_parameters = StdioServerParameters(
    command='uv',
    args=['run', 'server.py']
)

with ToolCollection.from_mcp(server_parameters, trust_remote_code=True) as tools:
    print('\n'.join(f"{t.name}: {t.description}" for t in tools))

We can also connect to an MCP server that is hosted on a remote machine. In this case, we can pass the `SSEServerParameters` to the `ToolCollection.from_mcp` method.

In [ ]:
from smolagents.mcp_client import MCPClient

with MCPClient(
    {'url': "https://abidlabs-mcp-tools.hf.space/gradio_api/mcp/sse"}
) as tools:
    # Tools from the remote server are available
    print('\n'.join(f"{t.name}: {t.description}" for t in tools))

Now we can use the MCP Client in a code agent:

In [ ]:
from smolagents import ToolCollection, CodeAgent, InferenceClientModel
from mcp.client.stdio import StdioServerParameters

model = InferenceClientModel()

server_parameters = StdioServerParameters(
    command='uv',
    args=['run', 'server.py']
)

with ToolCollection.from_mcp(
    server_parameters,
    trust_remote_code=True
) as tool_collection:
    agent = CodeAgent(
        model=model,
        tools=[*tool_collection.tools],
    )

    agent.run("What is the weather in Tokyo?")

We can also connect to an MCP packages. For example, to connect to the `pubmedmcp` package:

In [ ]:
from smolagents import ToolCollection, CodeAgent
from mcp import StdioServerParameters

server_parameters = StdioServerParameters(
    command='uv',
    args=['--quest', 'pubmedmcp@0.1.3'],
    env={'UV_PYTHON': "3.12", **os.environ}
)


with ToolCollection.from_mcp(
    server_parameters,
    trust_remote_code=True
) as tool_collection:
    agent = CodeAgent(
        tools=[*tool_collection.tools],
        add_base_tools=True
    )

    agent.run("Please find a remedy for hangover.")

## Gradio MCP Integration

Gradio allows developers to create UIs for their models with just a few of lines of Python code and useful for
- creating demos and prototypes
- sharing models with non-technical users
- testing and debugging moel behavior

With addition of MCP support, Gradio offers a straightforward way to expose AI model capabilities through the standardized MCP protocol.

In [ ]:
!pip install "gradio[mcp]"

We also need an LLM application that supports tool calling using the MCP protocol, such as Cursor (known as MCP Hosts).

### Creating an MCP Server with Gradio

In [ ]:
# demo.py

import gradio as gr

def letter_counter(word: str, letter: str) -> int:
    """Count the number of occurrences of a letter in a word or text.

    Args:
        word (str): The input text to search through
        letter (str): The letter to search for

    Returns:
        int: The number of times the letter appears in the text
    """
    word = word.lower()
    letter = letter.lower()
    count = word.count(letter)
    return count


# Create a standard Gradio interface
demo = gr.Interface(
    fn=letter_counter,
    inputs=['textbox', 'textbox'],
    outputs='number',
    title='Letter Counter',
    description="Enter text and a letter to count how many times the letter appears in the text."
)

# Launch both the Gradio web interface and the MCP server
if __name__ == "__main__":
    demo.launch(mcp_server=True)

Our letter counter function is now accessible through
- traditional Gradio web interface for direct human interaction,
- MCP Server that can be connected to compatible clients
    - the MCP server is accessible at: `http://your-server:port/gradio_api/mcp/sse`

### Gradio <==> MCP Integration

- **Tool conversion** - Each API endpoint in our Gradio app is automatically converted into an MCP tool with a corresponding name, description, and input schema. To view the tools and schemas, we can visit `http://your-server:port/gradio_api/mcp/schema` or go to the "View API" link in the footer of our Gradio app.

- **Environment variable support** - There are two ways to enable the MCP server functionality:
    - using the `mcp_server` parameter in `launch()`:
    ```python
    demo.launch(mcp_server=True)
    ```
    - using environment variables:
    ```bash
    export GRADIO_MCP_SERVER=True
    ```

- **File handling** - The server automatically handles file data conversions, including:
    - converting base64-encoded strings to file data
    - processing image files and returning them in the correct format
    - managing temporary file storage

- **Hosted MCP servers on HF Spaces** - We can publish our Gradio app for free on HuggingFace Spaces, which allows us to have a free hosted MCP server.

### Troublshooting

- **Type hints and docstring** - Ensure to provide type hints and valid docstrings for our functions. The docstring should include an "Args:" block with indented parameter names.
- **String inputs** - Accept input arguments as `str` and convert them to desired type inside the function.
- **SSE Support** - Some MCP Hosts do not support SSE-based MCP Servers, so we need to use `mcp-remote`:
    ```json
    {
        "mcpServers": {
            "gradio": {
                "command": "npx",
                "args: [
                    "mcp_remote",
                    "http://our-server:port/gradio_api/mcp/sse"
                ]
            }
        }
    }
    ```
- **Restart** - try restrating both our MCP Client and Server if encoutering connection issues.